In [1]:
import kagglehub
import os
from PIL import Image

# 1️⃣ Stáhnout dataset přes KaggleHub
dataset_path = kagglehub.dataset_download("farzadnekouei/top-view-vehicle-detection-image-dataset")

print("Path to dataset files:", dataset_path)

# 2️⃣ Výpis několika souborů ve složce datasetu
print("Prvních 10 souborů v datasetu:")
for file in os.listdir(dataset_path)[:10]:
    print(file)

# 3️⃣ Výpis složek a souborů
print("Složky v datasetu:")
print(os.listdir(dataset_path))

# 4️⃣ Najít první obrázek
first_image = None
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.lower().endswith((".jpg", ".png", ".jpeg")):
            first_image = os.path.join(root, file)
            break
    if first_image:
        break

if first_image:
    print("První obrázek:", first_image)

    # 5️⃣ Zobrazit obrázek
    img = Image.open(first_image)
    img.show()
else:
    print("Žádný obrázek nebyl nalezen v datasetu.")


Path to dataset files: C:\Users\ondra\.cache\kagglehub\datasets\farzadnekouei\top-view-vehicle-detection-image-dataset\versions\2
Prvních 10 souborů v datasetu:
Vehicle_Detection_Image_Dataset
Složky v datasetu:
['Vehicle_Detection_Image_Dataset']
První obrázek: C:\Users\ondra\.cache\kagglehub\datasets\farzadnekouei\top-view-vehicle-detection-image-dataset\versions\2\Vehicle_Detection_Image_Dataset\sample_image.jpg


In [2]:
import kagglehub
import os
import cv2
import numpy as np
import urllib.request
import time
import yt_dlp

# =====================
# === 1️⃣ YOLO SETUP ===
# =====================
weights = "yolov3.weights"
config = "yolov3.cfg"
labels_path = "coco.names"

def download_file(url, filename):
    """Stáhne soubor pouze pokud neexistuje."""
    if not os.path.exists(filename):
        print(f"[INFO] Stahuji {filename} ...")
        urllib.request.urlretrieve(url, filename)
        print(f"[INFO] {filename} stažen.")
    else:
        print(f"[INFO] {filename} už existuje, přeskočeno.")

# Stažení YOLO souborů
download_file("https://pjreddie.com/media/files/yolov3.weights", weights)
download_file("https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg", config)
download_file("https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names", labels_path)

# Načtení labelů
with open(labels_path, "r") as f:
    labels = [line.strip() for line in f.readlines()]

# Nastavení YOLO
net = cv2.dnn.readNet(weights, config)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(labels), 3))

# ==========================
# === 2️⃣ DETEKCE FUNKCE ===
# ==========================
def detect_objects(frame):
    height, width, channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    detected_objects = []
    class_ids, confidences, boxes = [], [], []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.4:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.4, 0.3)
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = labels[class_ids[i]]
            color = colors[class_ids[i]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            detected_objects.append(label)
    return frame, detected_objects

# ==========================
# === 3️⃣ ZPRACOVÁNÍ DATASETU ===
# ==========================
dataset_path = kagglehub.dataset_download("farzadnekouei/top-view-vehicle-detection-image-dataset")
os.makedirs("output", exist_ok=True)

image_files = []
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.lower().endswith((".jpg", ".png", ".jpeg")):
            image_files.append(os.path.join(root, file))

def process_dataset(images, max_images=5):
    print("[INFO] Zpracovávám Kaggle dataset...")
    for idx, img_path in enumerate(images[:max_images]):
        frame = cv2.imread(img_path)
        if frame is None:
            continue
        frame, objects = detect_objects(frame)
        output_path = os.path.join("output", f"dataset_{idx:03d}.jpg")
        cv2.imwrite(output_path, frame)
        print(f"[SAVE] {output_path} | Detekováno objektů: {len(objects)}")

process_dataset(image_files)

# ==========================
# === 4️⃣ ZPRACOVÁNÍ YOUTUBE STREAMU ===
# ==========================
YOUTUBE_URL = "https://www.youtube.com/watch?v=Xs_872Hah3o"
MAX_DURATION = 120  # sekundy

def get_youtube_stream(url):
    ydl_opts = {"format": "best", "quiet": True}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=False)
        return info["url"]

def process_youtube_stream(url, max_duration=120):
    try:
        stream_url = get_youtube_stream(url)
        cap = cv2.VideoCapture(stream_url)
        if not cap.isOpened():
            raise RuntimeError("Nelze otevřít YouTube stream.")
        print("[INFO] YouTube stream připraven.")
    except Exception as e:
        print(f"[WARN] YouTube stream nepřístupný: {e}")
        return

    start_time = time.time()
    frame_id = 0
    try:
        while True:
            if time.time() - start_time > max_duration:
                print(f"\n[STOP] Uplynulo {max_duration} sekund.")
                break

            ret, frame = cap.read()
            if not ret:
                time.sleep(1)
                continue

            frame, objects = detect_objects(frame)
            output_path = os.path.join("output", f"youtube_{frame_id:05d}.jpg")
            cv2.imwrite(output_path, frame)

            frame_id += 1
            if frame_id % 10 == 0:
                print(f"[INFO] Zpracováno {frame_id} snímků...")

            cv2.imshow("YouTube Detection", frame)
            if cv2.waitKey(1) == 27:  # ESC
                print("[STOP] Zastaveno uživatelem.")
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()

process_youtube_stream(YOUTUBE_URL, MAX_DURATION)

print("[INFO] Hotovo. Všechny výsledky uloženy ve složce 'output'.")


[INFO] Stahuji yolov3.weights ...
[INFO] yolov3.weights stažen.
[INFO] Stahuji yolov3.cfg ...
[INFO] yolov3.cfg stažen.
[INFO] coco.names už existuje, přeskočeno.
[INFO] Zpracovávám Kaggle dataset...
[SAVE] output\dataset_000.jpg | Detekováno objektů: 9
[SAVE] output\dataset_001.jpg | Detekováno objektů: 25
[SAVE] output\dataset_002.jpg | Detekováno objektů: 26
[SAVE] output\dataset_003.jpg | Detekováno objektů: 26
[SAVE] output\dataset_004.jpg | Detekováno objektů: 30
[INFO] YouTube stream připraven.
[INFO] Zpracováno 10 snímků...
[INFO] Zpracováno 20 snímků...
[INFO] Zpracováno 30 snímků...
[INFO] Zpracováno 40 snímků...
[INFO] Zpracováno 50 snímků...
[INFO] Zpracováno 60 snímků...
[INFO] Zpracováno 70 snímků...
[INFO] Zpracováno 80 snímků...
[INFO] Zpracováno 90 snímků...
[INFO] Zpracováno 100 snímků...
[INFO] Zpracováno 110 snímků...
[INFO] Zpracováno 120 snímků...
[INFO] Zpracováno 130 snímků...
[INFO] Zpracováno 140 snímků...
[INFO] Zpracováno 150 snímků...
[INFO] Zpracováno 160 

In [3]:
import os
import cv2
import numpy as np
import urllib.request
import time
import kagglehub
import yt_dlp

# =====================
# === 1️⃣ YOLO SETUP ===
# =====================
YOLO_DIR = "yolo_files"
os.makedirs(YOLO_DIR, exist_ok=True)

weights = os.path.join(YOLO_DIR, "yolov3.weights")
config = os.path.join(YOLO_DIR, "yolov3.cfg")
labels_path = os.path.join(YOLO_DIR, "coco.names")

# Soubory se nestahují do repozitáře, ale automaticky při běhu:
if not os.path.exists(weights):
    print("[INFO] Stahuji YOLO weights (poprvé může chvíli trvat)...")
    urllib.request.urlretrieve("https://pjreddie.com/media/files/yolov3.weights", weights)
if not os.path.exists(config):
    urllib.request.urlretrieve("https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg", config)
if not os.path.exists(labels_path):
    urllib.request.urlretrieve("https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names", labels_path)

with open(labels_path, "r") as f:
    labels = [line.strip() for line in f.readlines()]

net = cv2.dnn.readNet(weights, config)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(labels), 3))

# ==========================
# === 2️⃣ Kaggle Dataset ===
# ==========================
dataset_path = kagglehub.dataset_download("farzadnekouei/top-view-vehicle-detection-image-dataset")
os.makedirs("output", exist_ok=True)

image_files = []
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.lower().endswith((".jpg", ".png", ".jpeg")):
            image_files.append(os.path.join(root, file))

print(f"[INFO] Nalezeno {len(image_files)} obrázků v datasetu.")

# ==========================
# === 3️⃣ DETEKCE FUNKCE ===
# ==========================
def detect_objects(frame):
    height, width, _ = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    detected_objects = []
    class_ids, confidences, boxes = [], [], []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.4:
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(detection[0] * width - w / 2)
                y = int(detection[1] * height - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.4, 0.3)

    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = labels[class_ids[i]]
            color = colors[class_ids[i]]
            conf = confidences[i]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, f"{label} {conf:.2f}", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            detected_objects.append(label)

    return frame, detected_objects

# ==========================
# === 4️⃣ ZPRACOVÁNÍ DATASETU ===
# ==========================
print("[INFO] Zpracovávám Kaggle dataset...")
for idx, img_path in enumerate(image_files[:5]):  # prvních 5 obrázků
    frame = cv2.imread(img_path)
    if frame is None:
        continue
    frame, objects = detect_objects(frame)
    output_path = os.path.join("output", f"dataset_{idx:03d}.jpg")
    cv2.imwrite(output_path, frame)
    print(f"[SAVE] {output_path} | Detekováno objektů: {len(objects)}")

# ==========================
# === 5️⃣ YOUTUBE STREAM ===
# ==========================
YOUTUBE_URL = "https://www.youtube.com/watch?v=Xs_872Hah3o"
MAX_DURATION = 120
MAX_FRAMES = 50

def get_youtube_stream(url):
    ydl_opts = {"format": "best", "quiet": True}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=False)
        return info["url"]

try:
    stream_url = get_youtube_stream(YOUTUBE_URL)
    cap = cv2.VideoCapture(stream_url)
    if not cap.isOpened():
        raise RuntimeError("Nelze otevřít YouTube stream.")
    print("[INFO] YouTube stream připraven.")
except Exception as e:
    print(f"[WARN] YouTube stream nepřístupný: {e}")
    cap = None

if cap:
    print(f"[INFO] Zpracovávám YouTube stream (max {MAX_DURATION}s nebo {MAX_FRAMES} snímků).")
    start_time = time.time()
    frame_id = 0

    while True:
        if time.time() - start_time > MAX_DURATION or frame_id >= MAX_FRAMES:
            print(f"[STOP] Ukončuji po {frame_id} snímcích / {int(time.time()-start_time)} s.")
            break

        ret, frame = cap.read()
        if not ret:
            print("[WARN] Žádný snímek – čekám...")
            time.sleep(1)
            continue

        frame, objects = detect_objects(frame)
        output_path = os.path.join("output", f"youtube_{frame_id:05d}.jpg")
        cv2.imwrite(output_path, frame)

        frame_id += 1
        print(f"[INFO] Zpracováno {frame_id} snímků | Detekováno objektů: {len(objects)}")

        cv2.imshow("YouTube Detection", frame)
        if cv2.waitKey(1) == 27:  # ESC
            print("[STOP] Zastaveno uživatelem.")
            break

    cap.release()
    cv2.destroyAllWindows()

print("[INFO] Hotovo. Výsledky ve složce 'output'.")


[INFO] Stahuji YOLO weights (poprvé může chvíli trvat)...
[INFO] Nalezeno 627 obrázků v datasetu.
[INFO] Zpracovávám Kaggle dataset...
[SAVE] output\dataset_000.jpg | Detekováno objektů: 9
[SAVE] output\dataset_001.jpg | Detekováno objektů: 25
[SAVE] output\dataset_002.jpg | Detekováno objektů: 26
[SAVE] output\dataset_003.jpg | Detekováno objektů: 26
[SAVE] output\dataset_004.jpg | Detekováno objektů: 30
[INFO] YouTube stream připraven.
[INFO] Zpracovávám YouTube stream (max 120s nebo 50 snímků).
[INFO] Zpracováno 1 snímků | Detekováno objektů: 4
[INFO] Zpracováno 2 snímků | Detekováno objektů: 4
[INFO] Zpracováno 3 snímků | Detekováno objektů: 4
[INFO] Zpracováno 4 snímků | Detekováno objektů: 4
[INFO] Zpracováno 5 snímků | Detekováno objektů: 3
[INFO] Zpracováno 6 snímků | Detekováno objektů: 3
[INFO] Zpracováno 7 snímků | Detekováno objektů: 3
[INFO] Zpracováno 8 snímků | Detekováno objektů: 2
[INFO] Zpracováno 9 snímků | Detekováno objektů: 2
[INFO] Zpracováno 10 snímků | Deteková

In [6]:
import os
import cv2
import numpy as np
import urllib.request
import time
import kagglehub
import yt_dlp

# =========================
# === 1️⃣ YOLO SETUP ===
# =========================
def setup_yolo(weights="yolov3.weights", config="yolov3.cfg", labels_path="coco.names"):
    """Stáhne a načte YOLO model, pokud soubory chybí."""
    if not os.path.exists(weights):
        print("[INFO] Stahuji váhy YOLO...")
        urllib.request.urlretrieve("https://pjreddie.com/media/files/yolov3.weights", weights)
    if not os.path.exists(config):
        print("[INFO] Stahuji konfiguraci YOLO...")
        urllib.request.urlretrieve("https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg", config)
    if not os.path.exists(labels_path):
        print("[INFO] Stahuji COCO labels...")
        urllib.request.urlretrieve("https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names", labels_path)

    with open(labels_path) as f:
        labels = [line.strip() for line in f.readlines()]

    net = cv2.dnn.readNet(weights, config)
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    colors = np.random.uniform(0, 255, size=(len(labels), 3))
    return net, output_layers, labels, colors

# =========================
# === 2️⃣ DETEKCE ===
# =========================
def detect_objects(frame, net, output_layers, labels, colors, conf_threshold=0.4, nms_threshold=0.3):
    height, width, _ = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    boxes, confidences, class_ids = [], [], []
    for out in outs:
        for det in out:
            scores = det[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > conf_threshold:
                center_x = int(det[0] * width)
                center_y = int(det[1] * height)
                w = int(det[2] * width)
                h = int(det[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    detected_objects = []

    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = labels[class_ids[i]]
            conf = confidences[i]
            color = colors[class_ids[i]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, f"{label} {conf:.2f}", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            detected_objects.append(label)

    return frame, detected_objects

# =========================
# === 3️⃣ ZPRACOVÁNÍ DATASETU ===
# =========================
def process_dataset(dataset_path, net, output_layers, labels, colors, max_images=5):
    """Zpracuje obrázky z Kaggle datasetu a uloží výsledky do output/dataset/"""
    os.makedirs("output/dataset", exist_ok=True)
    image_files = []
    for root, _, files in os.walk(dataset_path):
        for f in files:
            if f.lower().endswith((".jpg", ".png", ".jpeg")):
                image_files.append(os.path.join(root, f))

    print(f"[INFO] Nalezeno {len(image_files)} obrázků v datasetu.")

    for idx, img_path in enumerate(image_files[:max_images]):
        frame = cv2.imread(img_path)
        if frame is None:
            continue
        frame, objects = detect_objects(frame, net, output_layers, labels, colors)
        out_path = os.path.join("output/dataset", f"{idx:03d}.jpg")
        cv2.imwrite(out_path, frame)
        print(f"[SAVE] {out_path} | Detekováno objektů: {len(objects)}")

# =========================
# === 4️⃣ ZPRACOVÁNÍ YOUTUBE STREAMU ===
# =========================
def get_youtube_stream(url):
    ydl_opts = {"format": "best", "quiet": True}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=False)
        return info["url"]

def process_youtube(url, net, output_layers, labels, colors, max_frames=50, max_duration=120, show=True):
    """Zpracuje stream YouTube a uloží snímky do output/youtube/"""
    os.makedirs("output/youtube", exist_ok=True)
    try:
        stream_url = get_youtube_stream(url)
        cap = cv2.VideoCapture(stream_url)
        if not cap.isOpened():
            raise RuntimeError("Nelze otevřít YouTube stream.")
        print("[INFO] YouTube stream připraven.")
    except Exception as e:
        print(f"[WARN] YouTube stream nepřístupný: {e}")
        return

    start_time = time.time()
    frame_id = 0
    while cap.isOpened():
        if time.time() - start_time > max_duration or frame_id >= max_frames:
            print(f"[STOP] Ukončuji po {frame_id} snímcích / {int(time.time()-start_time)} s.")
            break

        ret, frame = cap.read()
        if not ret:
            time.sleep(0.5)
            continue

        frame, objects = detect_objects(frame, net, output_layers, labels, colors)
        out_path = os.path.join("output/youtube", f"{frame_id:05d}.jpg")
        cv2.imwrite(out_path, frame)
        frame_id += 1
        print(f"[INFO] Zpracováno {frame_id} snímků | Detekováno objektů: {len(objects)}")

        if show:
            cv2.imshow("YouTube Detection", frame)
            if cv2.waitKey(1) == 27:
                print("[STOP] Zastaveno uživatelem.")
                break

    cap.release()
    if show:
        cv2.destroyAllWindows()

# =========================
# === 5️⃣ HLAVNÍ SPUŠTĚNÍ ===
# =========================
if __name__ == "__main__":
    net, output_layers, labels, colors = setup_yolo()
    dataset_path = kagglehub.dataset_download("farzadnekouei/top-view-vehicle-detection-image-dataset")
    process_dataset(dataset_path, net, output_layers, labels, colors)
    YOUTUBE_URL = "https://www.youtube.com/watch?v=Xs_872Hah3o"
    process_youtube(YOUTUBE_URL, net, output_layers, labels, colors)
    print("[INFO] Hotovo. Výsledky uloženy ve složce 'output/'.")


[INFO] Nalezeno 627 obrázků v datasetu.
[SAVE] output/dataset\000.jpg | Detekováno objektů: 0
[SAVE] output/dataset\001.jpg | Detekováno objektů: 0
[SAVE] output/dataset\002.jpg | Detekováno objektů: 0
[SAVE] output/dataset\003.jpg | Detekováno objektů: 0
[SAVE] output/dataset\004.jpg | Detekováno objektů: 0
[INFO] YouTube stream připraven.
[INFO] Zpracováno 1 snímků | Detekováno objektů: 0
[INFO] Zpracováno 2 snímků | Detekováno objektů: 0
[INFO] Zpracováno 3 snímků | Detekováno objektů: 0
[INFO] Zpracováno 4 snímků | Detekováno objektů: 0
[INFO] Zpracováno 5 snímků | Detekováno objektů: 0
[INFO] Zpracováno 6 snímků | Detekováno objektů: 0
[INFO] Zpracováno 7 snímků | Detekováno objektů: 0
[INFO] Zpracováno 8 snímků | Detekováno objektů: 0
[INFO] Zpracováno 9 snímků | Detekováno objektů: 0
[INFO] Zpracováno 10 snímků | Detekováno objektů: 0
[INFO] Zpracováno 11 snímků | Detekováno objektů: 0
[INFO] Zpracováno 12 snímků | Detekováno objektů: 0
[INFO] Zpracováno 13 snímků | Detekováno o

In [10]:
import os
import cv2
import numpy as np
import urllib.request
import time
import kagglehub
import yt_dlp

# =========================
# === 1️⃣ YOLO SETUP ===
# =========================
def setup_yolo(weights="models/yolov3.weights", config="models/yolov3.cfg", labels_path="models/coco.names"):
    """Stáhne a načte YOLO model, pokud chybí."""
    os.makedirs("models", exist_ok=True)

    files = {
        weights: "https://pjreddie.com/media/files/yolov3.weights",
        config: "https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg",
        labels_path: "https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names",
    }

    for file, url in files.items():
        if not os.path.exists(file):
            print(f"[INFO] Stahuji {os.path.basename(file)}...")
            urllib.request.urlretrieve(url, file)

    with open(labels_path) as f:
        labels = [line.strip() for line in f.readlines()]

    net = cv2.dnn.readNet(weights, config)
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    colors = np.random.uniform(0, 255, size=(len(labels), 3))
    return net, output_layers, labels, colors


# =========================
# === 2️⃣ DETEKCE ===
# =========================
def detect_objects(frame, net, output_layers, labels, colors, conf_threshold=0.4, nms_threshold=0.3):
    height, width, _ = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    boxes, confidences, class_ids = [], [], []
    for out in outs:
        for det in out:
            scores = det[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > conf_threshold:
                center_x = int(det[0] * width)
                center_y = int(det[1] * height)
                w = int(det[2] * width)
                h = int(det[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    detected_objects = []

    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = labels[class_ids[i]]
            conf = confidences[i]
            color = colors[class_ids[i]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, f"{label} {conf:.2f}", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            detected_objects.append(label)

    return frame, detected_objects


# =========================
# === 3️⃣ DATASET ===
# =========================
def process_dataset(dataset_path, net, output_layers, labels, colors, max_images=5):
    """Zpracuje obrázky z Kaggle datasetu a uloží výsledky do output/dataset/"""
    os.makedirs("output/dataset", exist_ok=True)
    image_files = [
        os.path.join(root, f)
        for root, _, files in os.walk(dataset_path)
        for f in files if f.lower().endswith((".jpg", ".png", ".jpeg"))
    ]

    print(f"[INFO] Nalezeno {len(image_files)} obrázků v datasetu.")

    for idx, img_path in enumerate(image_files[:max_images]):
        frame = cv2.imread(img_path)
        if frame is None:
            continue
        frame, objects = detect_objects(frame, net, output_layers, labels, colors)
        out_path = os.path.join("output/dataset", f"{idx:03d}.jpg")
        cv2.imwrite(out_path, frame)
        print(f"[SAVE] {out_path} | Detekováno objektů: {len(objects)}")


# =========================
# === 4️⃣ YOUTUBE STREAM ===
# =========================
def get_youtube_stream(url):
    ydl_opts = {"format": "best", "quiet": True}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=False)
        return info["url"]

def process_youtube(url, net, output_layers, labels, colors, max_frames=50, max_duration=120, show=True):
    """Zpracuje stream YouTube a uloží snímky do output/youtube/"""
    os.makedirs("output/youtube", exist_ok=True)
    try:
        stream_url = get_youtube_stream(url)
        cap = cv2.VideoCapture(stream_url)
        if not cap.isOpened():
            raise RuntimeError("Nelze otevřít YouTube stream.")
        print("[INFO] YouTube stream připraven.")
    except Exception as e:
        print(f"[WARN] YouTube stream nepřístupný: {e}")
        return

    start_time = time.time()
    frame_id = 0
    while cap.isOpened():
        if time.time() - start_time > max_duration or frame_id >= max_frames:
            print(f"[STOP] Ukončuji po {frame_id} snímcích / {int(time.time()-start_time)} s.")
            break

        ret, frame = cap.read()
        if not ret:
            time.sleep(0.5)
            continue

        frame, objects = detect_objects(frame, net, output_layers, labels, colors)
        out_path = os.path.join("output/youtube", f"{frame_id:05d}.jpg")
        cv2.imwrite(out_path, frame)
        frame_id += 1
        print(f"[INFO] Zpracováno {frame_id} snímků | Detekováno objektů: {len(objects)}")

        if show:
            cv2.imshow("YouTube Detection", frame)
            if cv2.waitKey(1) == 27:
                print("[STOP] Zastaveno uživatelem.")
                break

    cap.release()
    if show:
        cv2.destroyAllWindows()


# =========================
# === 5️⃣ MAIN ===
# =========================
if __name__ == "__main__":
    net, output_layers, labels, colors = setup_yolo()
    dataset_path = kagglehub.dataset_download("farzadnekouei/top-view-vehicle-detection-image-dataset")
    process_dataset(dataset_path, net, output_layers, labels, colors)
    YOUTUBE_URL = "https://www.youtube.com/watch?v=Xs_872Hah3o"
    process_youtube(YOUTUBE_URL, net, output_layers, labels, colors)
    print("[INFO] ✅ Hotovo. Výsledky jsou ve složce 'output/'.")


[INFO] Stahuji yolov3.weights...
[INFO] Stahuji yolov3.cfg...
[INFO] Stahuji coco.names...
[INFO] Nalezeno 627 obrázků v datasetu.
[SAVE] output/dataset\000.jpg | Detekováno objektů: 9
[SAVE] output/dataset\001.jpg | Detekováno objektů: 25
[SAVE] output/dataset\002.jpg | Detekováno objektů: 26
[SAVE] output/dataset\003.jpg | Detekováno objektů: 26
[SAVE] output/dataset\004.jpg | Detekováno objektů: 30
[INFO] YouTube stream připraven.
[INFO] Zpracováno 1 snímků | Detekováno objektů: 3
[INFO] Zpracováno 2 snímků | Detekováno objektů: 2
[INFO] Zpracováno 3 snímků | Detekováno objektů: 2
[INFO] Zpracováno 4 snímků | Detekováno objektů: 2
[INFO] Zpracováno 5 snímků | Detekováno objektů: 2
[INFO] Zpracováno 6 snímků | Detekováno objektů: 2
[INFO] Zpracováno 7 snímků | Detekováno objektů: 2
[INFO] Zpracováno 8 snímků | Detekováno objektů: 2
[INFO] Zpracováno 9 snímků | Detekováno objektů: 2
[INFO] Zpracováno 10 snímků | Detekováno objektů: 2
[INFO] Zpracováno 11 snímků | Detekováno objektů: 2

In [11]:
import os
import cv2
import numpy as np
import urllib.request
import time
import kagglehub
import yt_dlp

# =========================
# === 1️⃣ YOLO SETUP ===
# =========================
def setup_yolo(weights="models/yolov3.weights", config="models/yolov3.cfg", labels_path="models/coco.names"):
    """Stáhne a načte YOLO model, pokud chybí."""
    os.makedirs("models", exist_ok=True)

    files = {
        weights: "https://pjreddie.com/media/files/yolov3.weights",
        config: "https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg",
        labels_path: "https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names",
    }

    for file, url in files.items():
        if not os.path.exists(file):
            print(f"[INFO] Stahuji {os.path.basename(file)}...")
            urllib.request.urlretrieve(url, file)

    with open(labels_path) as f:
        labels = [line.strip() for line in f.readlines()]

    net = cv2.dnn.readNet(weights, config)
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    colors = np.random.uniform(0, 255, size=(len(labels), 3))
    return net, output_layers, labels, colors


# =========================
# === 2️⃣ DETEKCE ===
# =========================
def detect_objects(frame, net, output_layers, labels, colors, conf_threshold=0.4, nms_threshold=0.3):
    height, width, _ = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    boxes, confidences, class_ids = [], [], []
    for out in outs:
        for det in out:
            scores = det[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > conf_threshold:
                center_x = int(det[0] * width)
                center_y = int(det[1] * height)
                w = int(det[2] * width)
                h = int(det[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    detected_objects = []

    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = labels[class_ids[i]]
            conf = confidences[i]
            color = colors[class_ids[i]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, f"{label} {conf:.2f}", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            detected_objects.append(label)

    return frame, detected_objects


# =========================
# === 3️⃣ DATASET ===
# =========================
def process_dataset(dataset_path, net, output_layers, labels, colors, max_images=5):
    """Zpracuje obrázky z Kaggle datasetu a uloží výsledky do output/dataset/"""
    os.makedirs("output/dataset", exist_ok=True)
    image_files = [
        os.path.join(root, f)
        for root, _, files in os.walk(dataset_path)
        for f in files if f.lower().endswith((".jpg", ".png", ".jpeg"))
    ]

    print(f"[INFO] Nalezeno {len(image_files)} obrázků v datasetu.")

    for idx, img_path in enumerate(image_files[:max_images]):
        frame = cv2.imread(img_path)
        if frame is None:
            continue
        frame, objects = detect_objects(frame, net, output_layers, labels, colors)
        out_path = os.path.join("output/dataset", f"{idx:03d}.jpg")
        cv2.imwrite(out_path, frame)
        print(f"[SAVE] {out_path} | Detekováno objektů: {len(objects)}")


# =========================
# === 4️⃣ YOUTUBE STREAM ===
# =========================
def get_youtube_stream(url):
    ydl_opts = {"format": "best", "quiet": True}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=False)
        return info["url"]

def process_youtube(url, net, output_layers, labels, colors, max_frames=50, max_duration=120, show=True):
    """Zpracuje stream YouTube a uloží snímky do output/youtube/"""
    os.makedirs("output/youtube", exist_ok=True)
    try:
        stream_url = get_youtube_stream(url)
        cap = cv2.VideoCapture(stream_url)
        if not cap.isOpened():
            raise RuntimeError("Nelze otevřít YouTube stream.")
        print("[INFO] YouTube stream připraven.")
    except Exception as e:
        print(f"[WARN] YouTube stream nepřístupný: {e}")
        return

    start_time = time.time()
    frame_id = 0
    while cap.isOpened():
        if time.time() - start_time > max_duration or frame_id >= max_frames:
            print(f"[STOP] Ukončuji po {frame_id} snímcích / {int(time.time()-start_time)} s.")
            break

        ret, frame = cap.read()
        if not ret:
            time.sleep(0.5)
            continue

        frame, objects = detect_objects(frame, net, output_layers, labels, colors)
        out_path = os.path.join("output/youtube", f"{frame_id:05d}.jpg")
        cv2.imwrite(out_path, frame)
        frame_id += 1
        print(f"[INFO] Zpracováno {frame_id} snímků | Detekováno objektů: {len(objects)}")

        if show:
            cv2.imshow("YouTube Detection", frame)
            if cv2.waitKey(1) == 27:
                print("[STOP] Zastaveno uživatelem.")
                break

    cap.release()
    if show:
        cv2.destroyAllWindows()


# =========================
# === 5️⃣ MAIN ===
# =========================
if __name__ == "__main__":
    net, output_layers, labels, colors = setup_yolo()
    dataset_path = kagglehub.dataset_download("farzadnekouei/top-view-vehicle-detection-image-dataset")
    process_dataset(dataset_path, net, output_layers, labels, colors)
    YOUTUBE_URL = "https://www.youtube.com/watch?v=Xs_872Hah3o"
    process_youtube(YOUTUBE_URL, net, output_layers, labels, colors)
    print("[INFO] ✅ Hotovo. Výsledky jsou ve složce 'output/'.")


[INFO] Nalezeno 627 obrázků v datasetu.
[SAVE] output/dataset\000.jpg | Detekováno objektů: 9
[SAVE] output/dataset\001.jpg | Detekováno objektů: 25
[SAVE] output/dataset\002.jpg | Detekováno objektů: 26
[SAVE] output/dataset\003.jpg | Detekováno objektů: 26
[SAVE] output/dataset\004.jpg | Detekováno objektů: 30
[INFO] YouTube stream připraven.
[INFO] Zpracováno 1 snímků | Detekováno objektů: 3
[INFO] Zpracováno 2 snímků | Detekováno objektů: 3
[INFO] Zpracováno 3 snímků | Detekováno objektů: 3
[INFO] Zpracováno 4 snímků | Detekováno objektů: 2
[INFO] Zpracováno 5 snímků | Detekováno objektů: 2
[INFO] Zpracováno 6 snímků | Detekováno objektů: 2
[INFO] Zpracováno 7 snímků | Detekováno objektů: 2
[INFO] Zpracováno 8 snímků | Detekováno objektů: 2
[INFO] Zpracováno 9 snímků | Detekováno objektů: 2
[INFO] Zpracováno 10 snímků | Detekováno objektů: 3
[INFO] Zpracováno 11 snímků | Detekováno objektů: 2
[INFO] Zpracováno 12 snímků | Detekováno objektů: 2
[INFO] Zpracováno 13 snímků | Deteková